Intro
    This notebook serves as a demonstration to the data validation functions created for this project.
    There are three primary functions created that aim at validating data

    1) Validation of the data set. Checking for "hidden" issues in the data
        - Outliers
        - Dupplicate Data
        - Balance -- TODO
        - Correlation -- TODO

    2) Validation of the Train / Test Split
        -
        -
        -
        -
        -

    3) Creation of a validation data set
        -
        -


In [2]:
import pandas as pd
import numpy as  np

from scipy.stats import ks_2samp

In [93]:
def find_outliers(data_df, categorical_threshold=0.1):
    """
    :param data_df: Pandas DataFrame Containing the Data in question
    :param categorical_threshold: The frequency threshold to mark a categorical value as an outlier
    :return: A Pandas DataFrame listing the index of all outlier values

    This function cycles through the column in the provided data and looks for outliers
    There are two checks that are preformed.
        1) Checking if a categorical column contains any values that fall below the acceptable frequency threshold and thus an outlier
        2) Checking if a numerical column contains any statistical outliers, identified using IQR values
    """

    outlier_df = pd.DataFrame(columns=["Column","Value","Index","Issue"])
    for col in data_df.columns:
        print(data_df[col].dtype)
        if data_df[col].dtype == 'object' and all(isinstance(val, str) for val in data_df[col]):
            freq = data_df[col].value_counts(normalize=True)
            outlier_values = freq[freq < categorical_threshold].index
            outliers = test[test[col].isin(outlier_values)]
        elif data_df[col].dtype in ["float64","int64"]:
            q1 = data_df[col].quantile(0.25)
            q3 =  data_df[col].quantile(0.75)
            iqr = q3 - q1
            outliers = data_df[(data_df[col] < (q1 - 1.5 * iqr)) | (data_df[col] > (q3 + 1.5 * iqr))][col]
        else:
            pass

        # Prepare the temp "outliers" DF to be concatenated with the main outlier_df
        outliers = outliers.reset_index()
        outliers["Column"] = col
        outliers = outliers.rename(columns={"index":"Index",col:"Value"})
        outliers = outliers[["Column","Value","Index"]]
        outliers["Issue"] = "Outlier"
        outlier_df = pd.concat([outlier_df, outliers])

        return outlier_df

In [142]:
def find_duplicates(data_df):
    """
    :param data_df: Pandas DataFrame Containing the Data in question
    :return: A Pandas DataFrame listing the index of all duplicate values
    """

    duplicates = data_df[data_df.duplicated()]
    duplicates["Column"] = str(duplicates.columns.values)
    duplicates["Value"] = "N/A"
    duplicates = duplicates.reset_index()
    duplicates = duplicates.rename(columns={"index":"Index"})
    duplicates["Issue"] = "Duplicates"
    duplicates = duplicates[["Column","Value","Index","Issue"]]

    return duplicates

In [ ]:
def find_balance(data_df, label):
    pass

In [ ]:
def find_correlation(df):
    pass

In [150]:
def validate_dataset(data, categorical_threshold, data_type="excel"):

    # First step is to determine what was passed in and convert to a DF
    if data_type == "excel":
        data_df = pd.read_excel(data)
    elif data_type == "csv":
        data_df = pd.read_csv(data)
    elif data_type == "df":
        data_df = data
    else:
        return "Unexpected Type"

    # Check 1: Find Outliers
    outliers = find_outliers(data_df, categorical_threshold=categorical_threshold)

    # Check 2:
    duplicates = find_duplicates(data_df)

    # # Check 3:
    # balance = find_balance(data_df, label)
    #
    # # Check 4:
    # correlation = find_correlation(data_df)

    issues = pd.concat([outliers,duplicates])
    all_issues = issues.groupby("Index")["Issue"].unique().reset_index(name="Issues")

    return all_issues


In [ ]:
def test_representative(dataset, sample):
    return_df = pd.DataFrame(columns=["Feature","P_Value","Issue"])
    for feature in dataset.columns:
        ks_stat, p_value = ks_2samp(dataset[feature], sample[feature])
        # Distributions are not significantly different
        if p_value >= 0.05:
            pass
        # Distributions are significantly different
        else:
            return_df = pd.concat([return_df,pd.DataFrame([[feature,p_value,"Not Representative of Dataset"]],columns=["Feature","P_Value","Issue"])])

    return return_df

,Feature,P_Value,Issue
0,Test,0.4,Not Representative of Dataset


In [151]:
test = pd.read_csv("linear-regression-example/data/Advertising_data.csv")
# test = pd.read_csv("Machine-learning-without-any-libraries/6. LDA/Wine.csv")

In [152]:
test.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,2210
1,44.5,39.3,45.1,1040
2,17.2,45.9,69.3,930
3,151.5,41.3,58.5,1850
4,180.8,10.8,58.4,1290


In [153]:
test.dtypes

TV           float64
radio        float64
newspaper    float64
sales          int64
dtype: object

In [158]:
x = validate_dataset(test, categorical_threshold=0.1, data_type="df")
x

object


C:\Users\Caleb\AppData\Local\Temp\ipykernel_146188\3527661167.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates["Column"] = str(duplicates.columns.values)
C:\Users\Caleb\AppData\Local\Temp\ipykernel_146188\3527661167.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates["Value"] = "N/A"


,Index,Issues
0,5,[Duplicates]


In [157]:
import pandas as pd

# Create a sample dataframe with duplicate rows
data = {'col1': ['a', 'b', 'c', 'd', 'e', 'a'],
        'col2': [1, 2, 3, 4, 5, 1]}
test = pd.DataFrame(data)


In [144]:
find_duplicates(df)

C:\Users\Caleb\AppData\Local\Temp\ipykernel_146188\3527661167.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates["Column"] = str(duplicates.columns.values)
C:\Users\Caleb\AppData\Local\Temp\ipykernel_146188\3527661167.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates["Value"] = "N/A"


,Column,Value,Index,Issue
0,['col1' 'col2'],N/A,5,Duplicates
